# Working with timestamps and bursts

*This notebook is part of a [tutorial series](https://github.com/tritemio/FRETBursts_notebooks) for the [FRETBursts](http://tritemio.github.io/FRETBursts/) burst analysis software.*

> In this notebook show how to access different streams of timestamps,
> burst data (i.e. start and stop times and indexes, counts, etc...).
> This is useful for users wanting to access or process bursts data 
> and timestamps in custom ways.

# Load data

We start by loading the data, computing background and performing a standard burst search:

In [ ]:
from fretbursts import *
sns = init_notebook()

In [ ]:
filename = "./data/0023uLRpitc_NTP_20dT_0.5GndCl.hdf5"
d = loader.photon_hdf5(filename)
loader.alex_apply_period(d)
d.calc_bg(bg.exp_fit, time_s=30, tail_min_us='auto', F_bg=1.7)
d.burst_search()

# Getting the timestamps

To get the timestamps arrays for a given [photon stream](http://fretbursts.readthedocs.org/en/latest/ph_sel.html) we use [Data.get_ph_times](http://fretbursts.readthedocs.org/en/latest/data_class.html?highlight=get_ph_times#fretbursts.burstlib.Data.get_ph_times). Here a few example:

In [ ]:
ph = d.get_ph_times()                              # all the recorded photons
ph_dd = d.get_ph_times(ph_sel=Ph_sel(Dex='Dem'))   # donor excitation, donor emission
ph_d = d.get_ph_times(ph_sel=Ph_sel(Dex='DAem'))   # donor excitation, donor+acceptor emission
ph_aa = d.get_ph_times(ph_sel=Ph_sel(Aex='Aem'))   # acceptor excitation, acceptor emission

This are streams of all timestamps (both inside and outside the bursts).
Similarly, we can get "masks" of photons for each photon stream using 
[Data.get_ph_mask](http://fretbursts.readthedocs.org/en/latest/data_class.html?highlight=get_ph_mask#fretbursts.burstlib.Data.get_ph_mask):

In [ ]:
mask_dd = d.get_ph_mask(ph_sel=Ph_sel(Dex='Dem'))   # donor excitation, donor emission
mask_d = d.get_ph_mask(ph_sel=Ph_sel(Dex='DAem'))   # donor excitation, donor+acceptor emission
mask_aa = d.get_ph_mask(ph_sel=Ph_sel(Aex='Aem'))   # acceptor excitation, acceptor emission

Masks are arrays of booleans (True or False values) which are True
when the corresponding photon is in the stream. Note that all masks
have same number of elements as the all-photons timestamps array:

In [ ]:
ph.size, mask_dd.size, mask_d.size, mask_aa.size

Masks can be used to count photons in one stream:

In [ ]:
mask_d.sum()

and to obtain the timestamps for one stream:

In [ ]:
ph[mask_d]

Note that the arrays `ph[mask_d]` and `ph_d` are equal. This is an important point to understand.

# Burst data

There are several fields containing burst data:

**Start-stop**:

- `Data.mburst`: start-stop information for each burst

**Counts**:
- `Data.nd`: donor detetor counts during donor excitation
- `Data.na`: acceptor detetor counts during donor excitation
- `Data.naa`: acceptor detetor counts during acceptor excitation (ALEX only)
- `Data.nda`: donor detetor counts during acceptor excitation

**FRET**:
- `Data.E`: Proximity Ratio (when uncorrected) or FRET efficiency (when corrected)
- `Data.S`: "Stoichiometry" (ALEX only)

All these fields are lists containing one element per each channel.
In single-spot data these list have one single element which is accessed
using the `[0]` notation:

In [ ]:
bursts = d.mburst[0]
nd = d.nd[0]
na = d.na[0]
naa = d.naa[0]
E = d.E[0]
S = d.S[0]

## Burst start and stop

The start-stop burst data is in `bursts` (a variable of type [Bursts](), plural):

In [ ]:
bursts

Indexing `bursts` we can access a single burst:

In [ ]:
firstburst = bursts[0]
firstburst

The first two "columns" (both in `bursts` or `firstburst`) are the index of 
start and stop timestamps (relative to the all-photons timestamps).
The last two columns (`start` and `stop`) are the actual times of burst 
start and stop. To access any of these fields we type:

In [ ]:
bursts.istart

In [ ]:
firstburst.istart

Note that `ph[firstburst.istart]` is equal to `firstburst.start`:

In [ ]:
ph[firstburst.istart], firstburst.start

The same holds for stop:

In [ ]:
ph[firstburst.istop], firstburst.stop

Note that `bursts` is a `Bursts` object (plural) and `firstburst` is a `Burst` object (singular).
You can find more info on these objects in the documentation:

- [Low-level burst search functions](http://fretbursts.readthedocs.org/en/latest/burstsearch.html)

## Burst photon-counts

The variables `nd`, `na`, `naa` contains the number of photon in different photon streams.
By default these values are beckaground corrected and, if the correction coefficients 
are specified, are also corrected for leakage, direct excitation and gamma factor.

To get the raw counts before correction we can redo the burst search as follow:

In [ ]:
d.burst_search(computefret=False)
d.calc_fret(count_ph=True, corrections=False)

Note that if you select bursts, you also need to use `computefret=False`
to avoid recomputing E and S (which by defaults applyies the corrections):

In [ ]:
ds = d.select_bursts(select_bursts.size, th1=30, computefret=False)

In [ ]:
nd = ds.nd[0]
na = ds.na[0]
naa = ds.naa[0]
E = ds.E[0]
S = ds.S[0]

In [ ]:
nd

Note that the burst counts are integer values, confirming that the background
correction was not applied.

# See also

Thhis section of the documentation has more info how to use `Bursts` objects:

- [Low-level burst search functions](http://fretbursts.readthedocs.org/en/latest/burstsearch.html)

In particular, these 3 methods:

- [Bursts.recompute_times](http://fretbursts.readthedocs.org/en/latest/burstsearch.html?highlight=recompute_times#fretbursts.burstsearch.burstsearchlib.Bursts.recompute_times)
- [Bursts.recompute_index_expand](http://fretbursts.readthedocs.org/en/latest/burstsearch.html?highlight=recompute_index_expand#fretbursts.burstsearch.burstsearchlib.Bursts.recompute_index_expand)
- [Bursts.recompute_index_reduce](http://fretbursts.readthedocs.org/en/latest/burstsearch.html?highlight=recompute_index_reduce#fretbursts.burstsearch.burstsearchlib.Bursts.recompute_index_reduce)

Allow to transform burst data to refer a new timestamps arrays.